In [60]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import csv
import os

### Import FDs

In [103]:
filename_info = 'THE_WHEEL_12_31_2019_CHESTFINALSUBJECTS.xls'

pt_info = pd.read_excel(filename_info)
pt_info['label'] = pt_info['Group_Arb2'].map({0: 'control',
                                              1: 'pah',
                                              5: 'epah'})
pt_info['SID'] = pt_info['SID'].astype(str).str.zfill(3)

label_dict = dict(zip(pt_info['SID'].to_list(), pt_info['label'].to_list()))
label_dict

{'062': 'control',
 '063': 'control',
 '064': 'control',
 '065': 'control',
 '068': 'control',
 '070': 'control',
 '071': 'control',
 '072': 'control',
 '082': 'control',
 '087': 'control',
 '098': 'control',
 '100': 'control',
 '108': 'control',
 '109': 'control',
 '112': 'control',
 '115': 'control',
 '129': 'control',
 '145': 'control',
 '150': 'control',
 '183': 'control',
 '195': 'control',
 '200': 'control',
 '202': 'control',
 '216': 'control',
 '224': 'control',
 '225': 'control',
 '229': 'control',
 '237': 'control',
 '262': 'control',
 '267': 'control',
 '271': 'control',
 '276': 'control',
 '306': 'control',
 '316': 'control',
 '322': 'control',
 '323': 'control',
 '364': 'control',
 '013': 'pah',
 '020': 'pah',
 '021': 'pah',
 '026': 'pah',
 '028': 'pah',
 '030': 'pah',
 '036': 'pah',
 '044': 'pah',
 '059': 'pah',
 '097': 'pah',
 '111': 'pah',
 '116': 'pah',
 '125': 'pah',
 '131': 'pah',
 '139': 'pah',
 '158': 'pah',
 '169': 'pah',
 '170': 'pah',
 '173': 'pah',
 '184': 'pah

In [99]:
search_dir = os.path.join(os.getcwd(), 'FDs_data')

print(f'Performing search in:\n{search_dir}\n')

extension = 'csv'

csvFile_list = []
for root, dirs_list, files_list in os.walk(search_dir):
    for file_name in files_list:
        if file_name[-3:] == extension and \
        '-checkpoint' not in file_name and \
        'Visual' not in root:
            file_name_path = os.path.join(root, file_name)
            csvFile_list.append(file_name_path)

sid_list = pt_info['SID'].to_list()

csvFile_list = [csvFile for csvFile in csvFile_list
                if any(sid == csvFile.split('\\')[-1][:3] for sid in sid_list)]

len(csvFile_list)

Performing search in:
C:\Users\andre\Documents\GitHub\sampleNotebook\FDs_data



3792

In [106]:
lobe_list = ['left', 'right', 'whole']
artvein_list = ['', 'artery', 'vein']

df_list = []
for sid in sid_list:
    for lobe in lobe_list:
        for artvein in artvein_list:
            if artvein:
                files_matching = [csvFile for csvFile in csvFile_list
                                  if sid == csvFile.split('\\')[-1][:3] and \
                                     lobe in csvFile.split('\\')[-1] and \
                                     artvein.capitalize() in csvFile.split('\\')[-1]]
            else:
                files_matching = [csvFile for csvFile in csvFile_list
                                  if sid == csvFile.split('\\')[-1][:3] and \
                                     lobe in csvFile.split('\\')[-1] and \
                                     'Artery' not in csvFile.split('\\')[-1] and \
                                     'Vein' not in csvFile.split('\\')[-1]]
                
            df_dict = {
                'SID':             sid,
                'lobe':            lobe,
                'artery_or_vein':  artvein,
                'label':           label_dict[sid],
                'num_files':       len(files_matching),
                'files':           files_matching,
            }
            
            for f in files_matching:
                try:
                    with open(f) as csv_file:
                        csv_reader = csv.reader(csv_file)
                        csv_data = list(csv_reader)

                    folder = f.split('\\')[-2]

                    sc_arr = np.asarray(csv_data[4:]).astype(int)
                    sizes, counts = sc_arr[:, 0], sc_arr[:, 1]

                    sizes_log = sizes
                    counts_log = counts

                    # Calculate R Squared
                    correlation_matrix = np.corrcoef(sizes_log, counts_log)
                    r_squared = correlation_matrix[0, 1]**2
                    
                    df_dict[f'{folder}__Fractal_Dimension'] = float(csv_data[0][1])
                    df_dict[f'{folder}__Intercept'] =         float(csv_data[2][1])
                    df_dict[f'{folder}__R_Squared'] =         r_squared
                except:
                    print(f'ERROR:\n{f}\n')
            
            df_list.append(df_dict)
df = pd.DataFrame(df_list)
df.to_csv('test.csv', index=False)